In [1]:
# !pip install -q openai

In [2]:
# # For transformer models
# !pip install -q accelerate
# # !pip install -q bitsandbytes
# !pip install -i https://pypi.org/simple/ bitsandbytes
# # !pip install -q flash-attn --no-build-isolation

# # For sentence similarity
# !pip install sentence_transformers

# # For web queries
# !pip install googlesearch-python

# # For Retrieval Augmentated Generation (RAG) since HF doesn't have great support for it
# !pip install langchain
# !pip install chromadb

Looking in indexes: https://pypi.org/simple/
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/9b/63/489ef9cd7a33c1f08f1b2be51d1b511883c5e34591aaa9873b30021cd679/bitsandbytes-0.42.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 14.8 MB/s eta 0:00:0000:0100:01
  Obtaining dependency information for googlesearch-python from https://files.pythonhosted.org/packages/07/6d/bc3f2246868c29f1eacceed738057a7ea3c2b04737822b7fcc10128722b5/googlesearch_python-1.2.4-py3-none-any.whl.metadata


In [1]:
# !pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 870.8/870.8 kB 19.4 MB/s eta 0:00:00a 0:00:01


In [18]:
import anthropic

# if running locally
import os
import dotenv
dotenv.load_dotenv()

ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# if running on colab
# from google.colab import userdata
# ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

client = anthropic.Anthropic(
    api_key=ANTHROPIC_API_KEY,
)

In [110]:
claim_atomization_system = """You are a helpful assistant. Your task is to break down a set of statements given after <<<>>> into a minimal number of atomic claims.
These atomic claims need to be comprehensible, coherent, and context-independent. Return your output as a single, one-dimensional list of strings in square brackets.
Make sure each string is wrapped with double quotation marks (") and any quotes within with single (').
"""

claim_atomization_template = """
Segmentation Criteria:
1. The atomic claims should be context-independent, or should not rely on additional context to be understood.
2. Each atomic claim should focus on a single idea or concept regarding the truthfulness and/or plausibility of the statement. 
3. Aim for clarity and coherence in the segmented atomic claims. 
4. If a statement cannot be broken down further, return the entire statement as one atomic claim.

###
Here are some examples:
Statements: The Green New Deal proposed by Rep. Alexandria Ocasio-Cortez aims to achieve net-zero greenhouse gas emissions and create millions of green jobs.
Atomic Claims: ['The Green New Deal was proposed by Rep. Alexandria Ocasio-Cortez.', 'The Green New Deal aims to achieve net-zero greenhouse gas emissions.', 'The Green New Deal aims to create millions of green jobs.']
Statements: The Inflation Reduction Act, signed into law by President Biden in August 2022, is the largest investment in climate change mitigation in U.S. history.
Atomic Claims: ['The Inflation Reduction Act was signed into law by President Biden in August 2022.', 'The Inflation Reduction Act is the largest investment in climate change mitigation in U.S. history.']
Statements: President Trump's policies led to the highest inflation rate in over 40 years, resulting in economic hardship for millions of American families.
Atomic Claims: ["President Trump's policies led to the highest inflation rate in over 40 years.", "The high inflation rate caused by President Trump's policies resulted in economic hardship for millions of American families."]
###

You will only respond with the atomic claims in the format of a single, one-dimensional list of string objects in square brackets.
Do not provide any explanations or notes.

<<<
Statements: {statements}
>>>
Atomic Claims: []"""

def generate_atomic_claims(statements, num_examples=3):
  """
  Generates atomic claims for the input statements.

  Args:
      claim (str): The input statements.
      num_examples (int, optional): The number of few-shot examples to include in the prompt. Defaults to 3.

  Returns:
      str: The generated atomic claims.
  """
  prompt = claim_atomization_template.format(statements=statements)

  # claude output
  message = client.messages.create(
    model="claude-3-sonnet-20240229",
    max_tokens=1000,
    temperature=0,
    system=claim_atomization_system,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                }
            ]
        }
    ]
  )

  # Decode the generated text
  output_text = message.content[0].text

  # Extract only the list of claims from the model's output
  try:
      # Assuming output format directly returns Python list
      atomic_claims = eval(output_text.split('Atomic Claims:')[-1].strip())
      # POST-PROCESSING ERROR HANDLING: If list contains lists, return a flattened list
      if isinstance(atomic_claims[0], list):
          atomic_claims = [item for sublist in atomic_claims for item in sublist]
      return atomic_claims
  except:
      print(f"Error parsing model output: {output_text}")
      return ["Error parsing model output"]

# Example usage
statements = df.statement[37]
atomic_claims = generate_atomic_claims(statements)
print(f'statement: {statements}')
print(f'atomic claims: {atomic_claims}')

statement: "We know that President Biden didn’t just create this border crisis. He invited it with 94 executive actions in his first 100 days. When I took office, I took a different approach. I traveled to the Del Rio sector of Texas. That’s where I spoke to a woman who shared her story with me. She had been sex trafficked by the cartels starting at the age of 12. She told me not just that she was raped every day, but how many times a day she was raped. The cartels put her on a mattress in a shoe box of a room, and they sent men through that door over and over again for hours and hours on end. We wouldn’t be okay with this happening in a Third World country. This is the United States of America, and it is past time, in my opinion, that we start acting like it. President Biden’s border policies are a disgrace."
atomic claims: ['President Biden took 94 executive actions related to immigration in his first 100 days.', 'The speaker traveled to the Del Rio sector of Texas while in office.',

In [109]:
## Question Generation

# Define prompt template
question_generation_system = """
You are a helpful assistant. Let's break down the claim to questions step by step. Return your output as a single, one-dimensional list of strings in square brackets. 
"""
question_generation_template = """
Let's break down the claim to questions step by step.
Your task is to provide a set of unique, independent questions to search on the web to verify the claim given after <<<>>>.

Question generation criteria:
1. Each question should be related to the claim, context-independent, and be understood without access to the claim.
2. Each question should be able to be fact-checked by a True/False.
3. Be as specific and concise as possible. Try to minimize the number of questions.
4. Include enough details (e.g., pronoun specification, pronoun disambiguation) to ensure that the claim can be verified.

###
Here are some examples:
Claim: The high inflation rate caused by President Trump's policies resulted in economic hardship for millions of American families.
Questions: ["How did the high inflation rate under President Trump's policies affect American families?", "What economic challenges did millions of American families face due to the inflation rate under President Trump's policies?", 'What are the consequences of economic hardship caused by inflation under President Trump?']
Claim: President Trump's policies led to the highest inflation rate in over 40 years.
Questions: ["What impact did President Trump's policies have on the inflation rate?", "How does the inflation rate during President Trump's term compare to previous years?", "What is the significance of the inflation rate under President Trump's policies?"]
###

You will only respond with the generated questions in the format of a single, one-dimensional list in square-brackets. Do not provide any explanations or notes.

<<<
Claim: {claim}
>>>
Questions: []"""

def generate_questions(claim, num_examples=3):
    """
    Generates questions to verify the factuality of the input claim.

    Args:
        claim (str): The input claim.
        num_examples (int, optional): The number of few-shot examples to include in the prompt. Defaults to 3.

    Returns:
        str: The generated questions.
    """
    prompt = question_generation_template.format(claim=claim)

    # Print the entire prompt for debugging purposes
    # print(prompt)

    # claude output
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0,
        system=question_generation_system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )

    # Decode the generated text
    output_text = message.content[0].text

    # Extract only the list of questions from the model's output
    try:
        # Assuming output format directly returns Python list
        questions = eval(output_text.split('Questions:')[-1].strip())
        return questions
    except:
        print(f"Error parsing model output: {output_text}")
        return ["Error parsing model output"]

# Example usage for question generation
claim_question = dict()
for i, claim in enumerate(atomic_claims):
  questions = generate_questions(claim)
  claim_question[claim] = questions
  print(f"Claim: {claim}")
  print(f"Questions: {claim_question[claim]}")

Claim: In New York, there are no barriers to law enforcement to work with the federal government on immigration laws.
Questions: ['What are the laws or policies in New York regarding cooperation between state/local law enforcement and federal immigration authorities?', 'Are there any restrictions or limitations on New York law enforcement agencies assisting federal agencies in enforcing immigration laws?', "How does New York's stance on immigration enforcement cooperation with the federal government compare to other states?"]
Claim: In New York, there are 100 crimes where migrants can be handed over.
Questions: ['What is the number of crimes in New York where migrants can be handed over to immigration authorities?', 'What types of crimes in New York allow for the transfer of migrants to immigration authorities?', 'How does the policy of handing over migrants to immigration authorities for certain crimes in New York work?', 'What is the legal basis for handing over migrants to immigrati

In [83]:
## Web Querying & Scraping
import json
import requests
import pprint
import re
from bs4 import BeautifulSoup

SOURCE_BLACKLIST = ['politifact.org', 'factcheck.org', 'Snopes.com']

def extract_website_name(title):
    """Extracts the website name from a given title using regex"""
    delimiters = [' \| ', ' - ']
    for delimiter in delimiters:
        parts = re.split(delimiter, title)
        if len(parts) > 1:
            return parts[-1].strip()  # Return the last part of the split title
    # If no delimiters are found, return the original title
    return title.strip()

def scrape_text_from_website(url):
    """Scrapes text and metadata from a given website URL."""
    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')

            # Remove script and style tags
            for script in soup(["script", "style"]):
                script.decompose()

            # Extract all text from the website
            text = soup.get_text()

            # Clean up whitespace
            text = re.sub(r'\s+', ' ', text).strip()

            return text
        else:
            # print(f"Failed to retrieve content from the URL: {url}")
            return None
    except Exception as e:
        # print(f"Error during website scraping: {e}")
        return None

def fetch_search_results(question, scrape_website=True):
    """
    Fetches search results for a given question using an API.

    Args:
        question (str): The question to search for.
        scrape_website (bool, optional): Whether to scrape the website content. Defaults to False.

    Returns:
        list: A list of organic search results.
    """
    api_key = os.environ.get("SERPER_API_KEY")

    headers = {
        "X-API-KEY": api_key,
        "Content-Type": "application/json",
    }

    payload = json.dumps({"q": question})
    try:
        response = requests.post("https://google.serper.dev/search", headers=headers, data=payload)
        result = json.loads(response.text)

        # Extract the organic search results and transform them into our desired format
        results = []
        for item in result['organic']:
            # ALSO while iterating through the results, remove any websites on our source blacklist
            source = extract_website_name(item.get('link', ''))
            if source in SOURCE_BLACKLIST: continue
            website_text = scrape_text_from_website(item.get('link', '')) if scrape_website else item.get('snippet', '')
            if website_text is None or website_text == '': # if we failed to scrape the website, use the snippet
                website_text = item.get('snippet', '')
            results.append({
                "title": item.get('title', ''),
                "source": source,
                "date_published": item.get('date', ''),
                "relevant_excerpt": item.get('snippet', ''),
                "text": website_text,
                "search_position": item.get('position', -1),
                "url": item.get('link', ''),
            })
        return results

    except Exception as e:
        print(f"Failed to fetch information: {e}")
        return []

In [33]:
## RAG Retriever
from langchain.docstore.document import Document
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch

import copy

# Initialize embedding model for retrieval (sentence similarity)
BATCH_SIZE = 32
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
retriever_model_id='sentence-transformers/all-MiniLM-L6-v2'
retriever_model = HuggingFaceEmbeddings(
    model_name=retriever_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': BATCH_SIZE},
)

def retrieve_relevant_documents_using_rag(search_results, content_key, question, chunk_size=512, chunk_overlap=128, top_k=10):
    """
    Takes in search results and a query question, processes and splits the documents,
    and retrieves relevant documents using a RAG approach.

    Args:
        search_results (list of dict): A list of dictionaries containing web-scraped data.
        question (str): The query question for retrieving relevant documents.
        chunk_size (int): The maximum size of the text chunks.
        chunk_overlap (int): The overlap between consecutive text chunks.
        content_key (str): The key in the dictionary containing the text content.
        top_k (int): The number of relevant documents to retrieve.

    Returns:
        list: A list of relevant document chunks.
    """
    # Create LangChain documents from search results
    documents = []
    for result in search_results:
        page_content = result.pop(content_key, None)  # Extract the text content, remaining keys are metadata
        if page_content is not None:
            documents.append(Document(page_content=page_content, metadata=result))

    # Split documents into smaller chunks (if needed, based on document size)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    split_documents = text_splitter.split_documents(documents)

    # Initialize ChromaDB vector store to index the document chunks
    db = Chroma.from_documents(
        documents=split_documents,
        embedding=retriever_model,
    )

    # Retrieve the most relevant chunks for the given question
    relevant_docs = db.max_marginal_relevance_search(question, k=top_k)

    return relevant_docs

In [34]:
## RAG-based Question Answering

# Define prompt template
answer_synthesis_system = """
You are a helpful assistant. Answer the question and start your message with "Answer:"
"""

answer_synthesis_template = """
Let's think step by step. Your task is to synthesize the documents (along with their source metadata) provided below to answer the question given after <<<>>>.

Answer criteria:
1  Only use the documents below to answer the question.
2. Cite the relevant documents as JSON (including the source URL) after your answer starting with "Sources:"

If you cannot answer the question given the relevant documents, just say that you don't have enough information to answer the question. Do not make up an answer or sources.

Here are the relevant documents:
{documents}

<<<
Question: {question}
>>>
Answer: """

def synthesize_answer(relevant_docs, question, return_sources=True):
    """
    Synthesizes an answer to a given question using the relevant documents.

    Args:
        relevant_docs (list of dict): A list of relevant document chunks.
        question (str): The question to answer.

    Returns:
        str: The synthesized answer.
    """
    # Format the relevant documents for the prompt
    documents_text = ""
    for doc in relevant_docs:
        documents_text += f"Title: {doc.metadata.get('source_title', '')}\n"
        documents_text += f"URL: {doc.metadata.get('url', '')}\n"
        documents_text += f"Text: {doc.page_content.strip()}\n\n"
        documents_text += f"Date Published: {doc.metadata.get('date_published', '')}\n"

    # Fill in the prompt template with the relevant documents and the question
    prompt = answer_synthesis_template.format(documents=documents_text, question=question)
    prompt = prompt.replace('\n\n', '\n')

    # Print the entire prompt for debugging purposes
    # print(prompt)
    # print('-'*20)

    # claude output
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0,
        system=answer_synthesis_system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )

    # Decode the generated text
    output_text = message.content[0].text
    
    # Extract only the answer from the model's output
    try:
        answer = output_text.split('Answer:')[-1].split('Sources:')[0].strip()
        sources = output_text.split('Sources:')[-1].strip()
        if return_sources: return answer, sources
        return answer
    except:
        print(f"Error parsing model output: {output_text}")
        return "Error parsing model output"

In [35]:
## Claim Classification

# Define prompt template for reasoning and classification
# 1. -- from CRITIC
claim_classification_system = """
You are a logical reasoning assistant. In your response, return only one word after "Verdict:" and return an explanation after "Reasoning:".
"""

claim_classification_template = """
Given the original claim, a set of questions to help verify the claim, and their answers, reason step-by-step to come to a verdict on whether the claim is true or false. Think step-by-step about your reasoning process.
Return the verdict after "Verdict:" and provide a clear explanation after "Reasoning:"
For the verdict, only classify the claim as "True", "False", or "Unverifiable."

Reasoning Criteria:
1. Reason the claim over both plausibility and truthfulness.
2. Make sure to only reference to the question and answer pairs for your explanations.

Claim: {claim}

{questions_and_answers}

Verdict: (True/False/Unverifiable)

Reasoning: """

def classify_claim(claim, questions, answers, return_reasoning=True):
    """
    Uses a chain-of-thought approach to classify the original claim as true or false based on the answers to generated questions.

    Args:
        claim (str): The original claim.
        questions (list): List of questions related to the claim.
        answers (list): List of answers corresponding to the questions.

    Returns:
        str: The conclusion whether the claim is true or false with reasoning.
    """
    # Format the questions and answers into a single string
    questions_and_answers = ""
    for question, answer in zip(questions, answers):
        questions_and_answers += f"Question: {question}\nAnswer: {answer}\n\n"

    # Fill in the prompt template with the claim and formatted questions and answers
    prompt = claim_classification_template.format(claim=claim, questions_and_answers=questions_and_answers)

    # Print the entire prompt for debugging purposes
    # print(prompt)

    # claude output
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0,
        system=answer_synthesis_system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )

    # Decode the generated text
    output_text = message.content[0].text

    # Extract the verdict and reasoning separately from the model's output
    try:
        verdict = output_text.split('Verdict:')[-1].split('Reasoning:')[0].strip()
        reasoning = output_text.split('Reasoning:')[-1].strip()
        if return_reasoning: return verdict, reasoning
        return verdict
    except:
        raise ValueError(f"Error parsing model output: {output_text}")

    return output_text, verdict

In [76]:
from collections import Counter
import numpy as np
def generate_fact_score(verdicts):
    label = None
    perc_unverified = 0
    v_cleaned = verdicts
    if 'Unverifiable' in verdicts:
        v_cleaned = verdicts[:]
        v_cleaned.remove('Unverifiable')
        perc_unverified = Counter(verdicts)['Unverifiable'] / len(verdicts)
    perc_true = Counter(verdicts)['True'] / len(verdicts)
    perc_false = Counter(verdicts)['False'] / len(verdicts)
    perc = [perc_true, perc_false, perc_unverified]
    winner = np.argwhere(perc == np.amax(perc))
    # print(perc, winner)

    if len(winner) == 3: # three-way tie
        label = "Unverifiable"

    elif len(winner) == 2: # two-way tie
        if 0 in winner and 1 in winner: # half true
            label = 'Half True'
        elif 0 in winner and 2 in winner: # true & unverifable
            label = "Unverifiable"
        elif 1 in winner and 2 in winner: # false & unverifable
            label = "Unverifiable"

    elif len(winner) == 1:
        if 0 in winner:
            if perc_true == 1: # all true
                label = "True"
            elif Counter(v_cleaned)['True'] / len(v_cleaned) > 0.5: # mostly true
                label = "Mostly True"
            else:
                label = 'Unverifiable'
        elif 1 in winner:
            if perc_false == 1: # all false
                label = "Pants on Fire"
            elif Counter(v_cleaned)['False'] / len(v_cleaned) > 0.5: # mostly false
                label = "Mostly False"
            else:
                label = 'Unverifiable'
        elif 2 in winner:
            label = 'Unverifiable'
    return label

In [113]:
def verify_statement_claude(statement):
    atomic_claims = generate_atomic_claims(statement,0)
    print(len(atomic_claims))
    print(atomic_claims)
    output_dict = []
    verdicts = []
    reasonings = []
    for i, claim in enumerate(atomic_claims):
        claim_output = {}
        claim_output['claim'] = claim
        questions = generate_questions(claim,0)
        claim_output['qa-pairs'] = {}
        claim_output['qa-pairs']['questions'] = questions
        answers = []
        sources = []
        for j, question in enumerate(questions):
            search_results = fetch_search_results(question)
            relevant_docs = retrieve_relevant_documents_using_rag(search_results, 'relevant_excerpt', question)
            answer, source = synthesize_answer(relevant_docs, question)
            # output[f'claim_{i}']['questions'][f'answer_{j}'] = question
            answers.append(answer)
            sources.append(source)
        claim_output['qa-pairs']['answers'] = answers
        claim_output['qa-pairs']['sources'] = sources
        verdict, reasoning = classify_claim(claim, questions, answers)
        claim_output['verdict'] = verdict
        claim_output['reasoning'] = reasoning
        verdicts.append(verdict)
        reasonings.append((claim, reasoning))
        output_dict.append(claim_output)
    reasoning = '\n'.join(["Claim: " + item[0] + "\nReasoning: " + item[1] + "\n" for item in reasonings])
    return verdicts, output_dict, generate_fact_score(verdicts), reasoning

In [149]:
statement_verification_system = """
You are a fact-checking, logical-reasoning assistant. In your response, return only the label after "Verdict:" and return an explanation after "Reasoning:".
"""

statement_verification_template = """
Let's think step-by-step. Given a statement, verify whether the statement is factual by classify it to one of the five labels given (True/Mostly True/Half True/Mostly False/False/Unverifiable). 
Explain your reasoning in a logical manner and cite evidence wherever possible.

Reasoning Criteria:
1. Rate the statement over both plausibility and truthfulness.
2. If not enough information is provided, always err on the side of caution instead of blind guessing.

Statement: {statement}

Format your message in the exact format below.
Verdict: (True/Mostly True/Half True/Mostly False/False/Unverifiable)

Reasoning:
"""

def verify_statement_claude_statement_only(statement):
    # Fill in the prompt template with the relevant documents and the question
    prompt = statement_verification_template.format(statement=statement)

    # claude output
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0,
        system=statement_verification_system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    
    # Decode the generated text
    output_text = message.content[0].text

    try:
        verdict = output_text.split('Verdict:')[-1].split('Reasoning:')[0].strip()
        reasoning = output_text.split('Reasoning:')[-1].strip()
        return verdict, reasoning
    except:
        raise ValueError(f"Error parsing model output: {output_text}")
    

In [38]:
import pandas as pd
# modify base_dir
base_dir = '/Users/sunnyfang/Library/CloudStorage/GoogleDrive-yf2610@barnard.edu/My Drive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/LLM-FactChecker/data'
filename = '[FINAL] Pilot - Pilot Claims.csv'
full_path = os.path.join(base_dir, filename)
df = pd.read_csv(full_path)

In [39]:
df['statement'] = df['statement'].astype(str)

In [45]:
df['statement'][:50]

0     “The National Guard in the HISTORY of its life...
1     "On Jan. 6, 2021, U.S. Capitol 'protestors car...
2         "Not even one rocket (from Iran) hit Israel."
3     "326,000 migrants were flown to Florida with t...
4     "Crime is down in Venezuela by 67% because the...
5     "In New York, there are no barriers to law enf...
6     "Speaking of semiconductor industry jobs, "Kno...
7     "Starting in 2025 "no matter what your total b...
8     “Tens of thousands of auto jobs were lost nati...
9     "The current Congress is “the least productive...
10    "Video shows “New York Governor Kathy Hochul b...
11    "We’ve had 12 elections in 24 years in Wiscons...
12    After a 2022 law, “The vast majority of colleg...
13    "Insulin for Medicare beneficiaries "was costi...
14    “Support for Roe is higher today in America th...
15    "The 2022 CHIPS and Science Act “attracted $64...
16              “It is a fact that Obama created ISIS.”
17    "Millions of Arizonans will soon live unde

In [150]:
from tqdm.auto import tqdm
statement_only = []
for index, row in tqdm(df[:50].iterrows()):
    statement = row['statement']
    statement_only.append(verify_statement_claude_statement_only(statement))

0it [00:00, ?it/s]

In [151]:
claude_statement_only = pd.DataFrame(data = {
    "verdicts": [x[0] for x in statement_only],
    "reasonings": [x[1] for x in statement_only]})

In [152]:
claude_statement_only

,verdicts,reasonings
0,False,"The statement ""The National Guard in the HISTO..."
1,False,"The statement ""On Jan. 6, 2021, U.S. Capitol '..."
2,Unverifiable,"The statement ""Not even one rocket (from Iran)..."
3,False,"The statement that ""326,000 migrants were flow..."
4,False,"The statement ""Crime is down in Venezuela by 6..."
5,Mostly False,"The statement ""In New York, there are no barri..."
6,Mostly False,1. The statement claims that the average salar...
7,Mostly True,The statement appears to be referring to the c...
8,Unverifiable,"The statement makes two claims: 1) ""Tens of th..."
9,Unverifiable,"The statement ""The current Congress is 'the le..."


In [154]:
# modify base_dir
base_dir = '/Users/sunnyfang/Library/CloudStorage/GoogleDrive-yf2610@barnard.edu/My Drive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/'
filename = 'final-claude-3-sonnet-baseline.xlsx'
full_path = os.path.join(base_dir, filename)
claude_statement_only.to_excel(full_path)

filename = 'claude_baseline.csv'
full_path = os.path.join(base_dir, filename)
claude_statement_only.to_csv(full_path)

In [41]:
fact_check_samples = []
for index, row in tqdm(df[:1].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

2
What is the role of the National Guard in responding to disasters?
Failed to retrieve content from the URL: https://www.jstor.org/stable/40924733
Failed to retrieve content from the URL: https://sakeraviation.com/the-role-of-the-air-national-guard-when-natural-disasters-strike/
Under what circumstances is the National Guard deployed after a disaster?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.jstor.org/stable/40924733


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


When is the National Guard typically deployed?
Failed to retrieve content from the URL: https://www.army.nationalguard.mil/About-Us/Army-Guard-Mission/
What are the circumstances under which the National Guard is called in?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Does the National Guard respond only after a disaster has occurred or can they be deployed in anticipation of potential disasters?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [46]:
for index, row in tqdm(df[1:5].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

1
Were any weapons carried by the individuals who entered the U.S. Capitol on January 6, 2021?
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.poynter.org/fact-checking/2024/january-6-capitol-no-weapons-firearms-guns-evidence/
Failed to retrieve content from the URL: https://www.everytown.org/were-guns-present-at-the-january-6th-capitol-insurrection/
Failed to retrieve content from the URL: https://www.newsweek.com/fact-check-were-there-armed-protesters-capitol-january-6-1715326
What types of weapons, if any, were carried by the individuals who entered the U.S. Capitol on January 6, 2021?
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://everytownresearch.org/report/armed-extremism-us-capitol/
Were the individuals who entered the U.S.

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Were migrants flown or transported by other means to Florida using public funds?
Failed to retrieve content from the URL: https://www.migrationpolicy.org/article/migrant-asylum-seeker-busing
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
What was the purpose of transporting migrants to Florida using taxpayer dollars?
Failed to retrieve content from the URL: https://www.axios.com/2023/11/25/migrant-buses-immigrants-spend-taxpayer-millions
Failed to retrieve content from the URL: https://www.newsweek.com/fact-check-did-ron-desantis-flying-migrants-martha-vineyard-cost-12m-1743291
What is President Joe Biden's secret migrant flight program?
Error during website scraping: HTTPSConnectionPool(host='www.usnews.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://cis.org/Bensman/Secret-Finally-Revealed-Americans-Can-Know-US-Cities-Receiving-Hundreds-Thousands
What evide

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Is Venezuela sending gangs and criminals to the United States?
Failed to retrieve content from the URL: https://cis.org/Arthur/Venezuela-Sending-Violent-Criminals-United-States


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.ktsm.com/news/fbi-agent-others-warn-of-venezuelan-gang-entering-into-us/
Failed to retrieve content from the URL: https://www.state.gov/reports/2022-country-reports-on-human-rights-practices/venezuela/
What evidence exists of Venezuela depositing gangs and criminals into the United States?
Failed to retrieve content from the URL: https://cis.org/Arthur/Venezuela-Sending-Violent-Criminals-United-States
Failed to retrieve content from the URL: https://www.ktsm.com/news/fbi-agent-others-warn-of-venezuelan-gang-entering-into-us/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


How are gangs and criminals from Venezuela allegedly being transported to the United States?
Failed to retrieve content from the URL: https://www.ktsm.com/news/fbi-agent-others-warn-of-venezuelan-gang-entering-into-us/
Failed to retrieve content from the URL: https://cis.org/Arthur/Venezuela-Sending-Violent-Criminals-United-States
Failed to retrieve content from the URL: https://www.gmtoday.com/news/national/crime-concerns-grow-as-members-of-a-violent-venezuelan-gang-arrive-in-american-cities/article_1b8c456a-f680-11ee-af2e-4707bda946fe.html
Error during website scraping: HTTPSConnectionPool(host='www.miamiherald.com', port=443): Read timed out. (read timeout=5)
What is the scale or extent of Venezuela allegedly sending gangs and criminals to the United States?
Failed to retrieve content from the URL: https://cis.org/Arthur/Venezuela-Sending-Violent-Criminals-United-States


In [56]:
for index, row in tqdm(df[5:10].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

2
What are the laws or policies in New York regarding cooperation between state/local law enforcement and federal immigration authorities?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Are there any restrictions or limitations on New York law enforcement agencies assisting federal agencies in enforcing immigration laws?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

How does the current Congress's productivity level impact governance and policymaking?
Failed to retrieve content from the URL: https://bipartisanpolicy.org/wp-content/uploads/2019/03/BPC-Economy-Congress-and-Evidence-Based-Policymaking.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://crsreports.congress.gov/product/pdf/IF/IF10557
Failed to retrieve content from the URL: https://www.jstor.org/stable/45132494


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://platforms.aei.org/the-governance-of-american-innovation-restoring-congresss-role/


In [60]:
for index, row in tqdm(df[10:15].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

1
Did a video capture New York Governor Kathy Hochul being denied entry into the wake for slain NYPD officer Jonathan Diller?
Failed to retrieve content from the URL: https://www.newsweek.com/kathy-hochul-video-jonathan-diller-nypd-wake-1885248
Failed to retrieve content from the URL: https://www.politico.com/news/2024/03/30/hochul-wake-nypd-diller-00149839
What was the reason given for denying Governor Kathy Hochul entry into the wake for Officer Jonathan Diller?
Failed to retrieve content from the URL: https://www.politico.com/news/2024/03/30/hochul-wake-nypd-diller-00149839
Failed to retrieve content from the URL: https://www.newsweek.com/kathy-hochul-video-jonathan-diller-nypd-wake-1885248
Was Governor Kathy Hochul ultimately allowed to attend the wake for Officer Jonathan Diller?
Failed to retrieve content from the URL: https://pix11.com/news/local-news/gov-hochul-confronted-while-leaving-wake-for-nypd-officer/
How did Governor Kathy Hochul respond to being denied entry into the w

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


How many of those elections in Wisconsin were decided by a margin of less than 30,000 votes?
Failed to retrieve content from the URL: https://abcnews.go.com/Politics/wireStory/democrats-hope-biden-boost-ballot-races-wisconsins-new-108691070
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
What is the significance of elections being decided by a small margin of votes in Wisconsin?
What factors contribute to close election results in Wisconsin?
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.nytimes.com/2020/04/07/us/politics/wisconsin-primary-election.html
Error during website scraping: HTTPSConnectionPool(host='www.usnews.com', port=443): Read timed out. (read timeout=5)
1
What is the impact of the 2022 law on the availability of on-campus poll sites in New York State colleges?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

What policy or program has led to the $35 monthly cost of insulin for Medicare beneficiaries?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


How does the $35 monthly cost of insulin impact Medicare beneficiaries financially?
Failed to retrieve content from the URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9862367/
Failed to retrieve content from the URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7646207/
1
What is the current level of support for Roe v. Wade in America?
Failed to retrieve content from the URL: https://www.axios.com/2023/07/12/most-americans-support-abortion-poll
Failed to retrieve content from the URL: https://www.americanprogress.org/article/a-year-after-the-supreme-court-overturned-roe-v-wade-trends-in-state-abortion-laws-have-emerged/
How does the current support for Roe v. Wade compare to historical levels?
Failed to retrieve content from the URL: https://www.americanprogress.org/article/a-year-after-the-supreme-court-overturned-roe-v-wade-trends-in-state-abortion-laws-have-emerged/
Failed to retrieve content from the URL: https://www.reuters.com/world/us/how-abortion-became-divisive-issue-us-pol

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://reproductiverights.org/roe-v-wade/
How is public opinion on Roe v. Wade measured and tracked?
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9777967/
Failed to retrieve content from the URL: https://www.nytimes.com/2023/06/23/us/roe-v-wade-abortion-views.html


In [72]:
for index, row in tqdm(df[12:13].iterrows()):
    statement = row['statement']
    fact_check_samples[12] = verify_statement_claude(statement)

0it [00:00, ?it/s]

1
What is the impact of the 2022 law on the availability of on-campus poll sites in New York State colleges?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.timesunion.com/opinion/article/commentary-new-york-mandated-on-campus-voting-18691817.php
Failed to retrieve content from the URL: https://www.democracydocket.com/news-alerts/bard-college-calls-out-lack-of-polling-sites-on-new-york-college-campuses/
Failed to retrieve content from the URL: https://elections.ny.gov/system/files/documents/2023/10/2023-new-york-state-election-law-update.pdf
How many colleges in New York State currently have on-campus poll sites after the 2022 law?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Failed to retrieve content from the URL: https://www.timesunion.com/opinion/article/commentary-new-york-mandated-on-campus-voting-18691817.php
Failed to retrieve content from the URL: https://www.democracydocket.com/news-alerts/bard-college-calls-out-lack-of-polling-sites-on-new-york-college-campuses/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


What are the potential consequences of the lack of on-campus poll sites for college students in New York State?


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.timesunion.com/opinion/article/commentary-new-york-mandated-on-campus-voting-18691817.php
Failed to retrieve content from the URL: https://www.democracydocket.com/news-alerts/bard-college-calls-out-lack-of-polling-sites-on-new-york-college-campuses/
Failed to retrieve content from the URL: https://www.findlaw.com/voting/how-do-i-protect-my-right-to-vote-/voter-suppression-and-college-students.html
Failed to retrieve content from the URL: https://www.nytimes.com/2020/10/20/us/politics/college-students-voting-location.html


In [75]:
for index, row in tqdm(df[15:20].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

1
What is the total amount of private investment attracted by the 2022 CHIPS and Science Act?
Failed to retrieve content from the URL: https://www.semiconductors.org/the-chips-act-has-already-sparked-200-billion-in-private-investments-for-u-s-semiconductor-production/
Failed to retrieve content from the URL: https://www.congress.gov/bill/117th-congress/house-bill/4346
Failed to retrieve content from the URL: https://www.state.gov/the-u-s-department-of-state-international-technology-security-and-innovation-fund/
How much private investment was generated as a result of the 2022 CHIPS and Science Act?
Failed to retrieve content from the URL: https://www.semiconductors.org/the-chips-act-has-already-sparked-200-billion-in-private-investments-for-u-s-semiconductor-production/
Failed to retrieve content from the URL: https://www.vanhollen.senate.gov/download/chips-and-science-act-of-2022-summary&download=1
What is the significance of the $640 billion in private investment attracted by the 202

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9836943/
Failed to retrieve content from the URL: https://jamanetwork.com/journals/jamanetworkopen/fullarticle/2814017
Failed to retrieve content from the URL: https://www.americanprogress.org/article/abortion-bans-will-result-in-more-women-dying/
What safeguards or provisions are in place in the abortion ban to protect women's health?
Failed to retrieve content from the URL: https://www.congress.gov/bill/117th-congress/house-bill/3755/text
Failed to retrieve content from the URL: https://reproductiverights.org/the-womens-health-protection-act-federal-legislation-to-protect-the-right-to-access-abortion-care/
Failed to retrieve content from the URL: https://www.congress.gov/bill/118th-congress/senate-bill/701
Failed to retrieve content from the URL: https://actforwomen.org/whpa-faqs/
What are the exceptions or provisions for cases of rape or incest in the abortion ban?
Failed to retrieve content from t

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
What language or terminology is used in the statement to refer to illegal immigrants?
Failed to retrieve content from the URL: https://medium.com/immigration-in-america/the-language-of-illegal-immigration-1178594f0930
Error during website scraping: HTTPSConnectionPool(host='www.vera.org', port=443): Max retries exceeded with url: /news/no-person-is-illegal-the-language-we-use-for-immigration-matters (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Failed to retrieve content from the URL: https://www.unhcr.org/cy/wp-content/uploads/sites/41/2018/09/TerminologyLeaflet_EN_PICUM.pdf
Does the statement portray illegal immigrants in a militaristic or hostile manner?
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=4

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


2
Did Rep. Adam Schiff vote to allocate $100 billion in funding for foreign countries?
Failed to retrieve content from the URL: https://www.politico.com/live-updates/2023/12/06/congress/senate-gop-blocks-foreign-aid-ukraine-border-00130387
Failed to retrieve content from the URL: https://www.congress.gov/member/adam-schiff/S001150?q=%7B%22status%22%3A%22House+amendment+offered%22%7D
Failed to retrieve content from the URL: https://www.congress.gov/bill/117th-congress/house-bill/2617
Failed to retrieve content from the URL: https://www.opensecrets.org/members-of-congress/adam-schiff/summary?cid=N00009585
What specific legislation or budget proposal involved Rep. Adam Schiff voting to send $100 billion to foreign countries?
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.congress.gov/bill/118th-congress/senate-bill/2226
Failed to retrieve content from the URL

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


What is the current national debt of the United States?
Failed to retrieve content from the URL: https://www.usdebtclock.org/
How does the $35 trillion figure compare to the national debt figures from previous years?
What factors have contributed to the accumulation of the $35 trillion national debt?
Failed to retrieve content from the URL: https://www.americanprogress.org/article/tax-cuts-are-primarily-responsible-for-the-increasing-debt-ratio/
Failed to retrieve content from the URL: https://www.thepolicycircle.org/brief/fiscal-responsibility/
What are the potential consequences of a high national debt for the United States?
Failed to retrieve content from the URL: https://carnegieendowment.org/chinafinancialmarkets/86397


In [82]:
for index, row in tqdm(df[20:25].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

3
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://thehill.com/homenews/media/4494066-abc-news-president-trump-black-people-racist/
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://gen.medium.com/trump-keeps-saying-racist-things-heres-the-ever-growing-list-of-examples-21774f6749a4
Failed to retrieve content from the URL: https://www.reuters.com/world/us/bloodbath-vermin-animals-trumps-rhetoric-trail-2024-03-22/
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com',

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.sciencedirect.com/science/article/pii/S1198743X15301348
Failed to retrieve content from the URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4735229/
Failed to retrieve content from the URL: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5055265/
Error during website scraping: HTTPSConnectionPool(host='www.tga.gov.au', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.sciencedirect.com/topics/medicine-and-dentistry/pharmaceutical-care
Failed to retrieve content from the URL: https://www.oecd.org/health/pharmaceuticals.htm
Failed to retrieve content from the URL: https://onlinelibrary.wiley.com/doi/pdf/10.1016/S1526-9523(02)00323-9
Failed to retrieve content from the URL: https://onlinelibrary.wiley.com/doi/full/10.1016/S1526-9523(02)00323-9
Failed to retrieve content from the URL: https://www.unicef.org/parenting/health/parents-frequently-asked-questions-vaccines
Error during website scra

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

Failed to retrieve content from the URL: https://www.pnas.org/doi/full/10.1073/pnas.2015762117
2
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.nytimes.com/2016/07/21/us/politics/donald-trump-issues.html
Failed to retrieve content from the URL: https://www.politico.com/news/2024/02/14/trump-nato-allies-00141590
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.nytimes.com/2024/02/10/us/politics/trump-nato-russia.html
Error during website scraping: HTTPSConnectionPool(host='www.usnews.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.reuters.com/world/what-did-trump-say-about-nato-funding-what-is-article-5-2024-02-12/
Failed to retrieve content from the URL: https://thehill.com/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://www.jstor.org/stable/48732276


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Failed to retrieve content from the URL: https://theconversation.com/what-a-second-trump-presidency-might-mean-for-the-rest-of-the-world-227641
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.nytimes.com/2016/07/21/us/politics/donald-trump-issues.html
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
Failed to retrieve content from the URL: https://www.reuters.com/world/us/trump-comments-russia-nato-appalling-unhinged-white-house-spokesperson-2024-02-11/
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=5)
1
Failed to retrieve content from the URL: https://thehill.com/homenews/house/4640096-johnson-george-soros-campus-protests/
Error during website scraping: HTTPSConnectionPool(host='www.washingtonpost.com', po

In [86]:
for index, row in tqdm(df[24:25].iterrows()):
    statement = row['statement']
    fact_check_samples[24] = verify_statement_claude(statement)

0it [00:00, ?it/s]

2


In [90]:
for index, row in tqdm(df[25:30].iterrows()):
    print(index)
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))

0it [00:00, ?it/s]

25
1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


26
2


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


27
1
28
2


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


29
1


In [94]:
for index, row in tqdm(df[30:35].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))
    print(f"done with row {index}! ")

0it [00:00, ?it/s]

2
done with row 30! 
1
done with row 31! 
1


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


done with row 32! 
2
done with row 33! 
1
done with row 34! 


In [95]:
for index, row in tqdm(df[35:40].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))
    print(f"done with row {index}! ")

0it [00:00, ?it/s]

3


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


done with row 35! 
3


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

done with row 37! 
5


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


done with row 38! 
3


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


done with row 39! 


In [137]:
from tqdm.auto import tqdm

for index, row in tqdm(df[40:50].iterrows()):
    statement = row['statement']
    fact_check_samples.append(verify_statement_claude(statement))
    print(f"done with row {index}! ")

0it [00:00, ?it/s]

4
['We had the best unemployment rates ever.', "The unemployment rates were real unemployment, not like today where nobody's working and they consider it to be.", 'The current unemployment rate calculation is too complicated to explain.', 'The current unemployment rate calculation is a whole different thing from the previous calculation.']


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

done with row 40! 
4
['House Republicans took numerous votes.', 'The votes taken by House Republicans would have damaged economic growth.', 'The votes taken by House Republicans would have harmed national security.', 'House Republicans attempted to eliminate over 2,000 Border Patrol agents.']


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

done with row 46! 
1
["This trial that I have now, that's a Biden trial."]


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


done with row 47! 
2
['President Biden is the first candidate in history that has used the federal agencies to censor political speech.', 'President Biden is the first president in history that has used the federal agencies to censor his opponent.']


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


done with row 48! 
1
["Dollar Tree, Walgreens, Macy's, Foot Locker, Gap, Burger King, Regal Cinemas and Kroger have had mass store closures during Joe Biden's presidency."]
done with row 49! 


In [ ]:
claude_sample = pd.DataFrame(data = {
    "verdicts": [x[0] for x in fact_check_samples],
    "fact_score": [x[2] for x in fact_check_samples],
    "output": [x[1] for x in fact_check_samples],
    "reasonings": [x[3] for x in fact_check_samples]})

In [29]:
[x[0] for x in fact_check_samples]

[['Based on the provided documents, the claim "The National Guard gets called in after a disaster, not before something happens" is false.\n\nThe documents clearly indicate that the National Guard has both a responsive role in providing assistance after disasters and emergencies occur, as well as a preventative role in helping prepare for and mitigate potential disasters before they happen.\n\nSpecifically:\n\n1. The document from the U.S. Army Command and General Staff College states that the National Guard "routinely deploys to assist states with preparing for disasters like wildfires, floods, earthquakes, and oil spills, in addition to responding after they occur."\n\n2. The article from Virginia Commonwealth University notes that "The National Guard can play a vital role in saving lives, stemming suffering and protecting property before and after disasters strike."\n\n3. The Department of Defense website mentions that the National Guard can "help prepare" in addition to responding 

In [156]:
# modify base_dir
base_dir = '/Users/sunnyfang/Library/CloudStorage/GoogleDrive-yf2610@barnard.edu/My Drive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/'
filename = 'final-claude-3-sonnet-samples.xlsx'
full_path = os.path.join(base_dir, filename)
claude_sample.to_excel(full_path)

filename = 'claude_complete.csv'
full_path = os.path.join(base_dir, filename)
claude_sample.to_csv(full_path)

In [1]:
fact_checking_system = """
You are a fact-checking, logical-reasoning assistant. 
"""

fact_checking_prompt = """
Let's think step-by-step. 
I will give you a statement and a set of claims derived from that statement. 
Each claim is followed by a verdict on their truthfulness (True/False/Unverifiable), and reasoning for each. 
Please verify whether the original statement given above is factual by classifying it to one of the five following labels: (True/Mostly True/Half True/Mostly False/False/Unverifiable). 

Explain your reasoning in a logical manner and cite evidence wherever possible.

Reasoning Criteria:
1. Rate the statement over both plausibility and truthfulness.
2. If not enough information is provided, always err on the side of caution instead of blind guessing.

Statement: {statement}

{claim_reasoning_pair}

In your response, return only the label after "Verdict:" and return an explanation after "Reasoning:".
"""
def verify_statement_claude_claim_reasoning(statement, reasoning):
    # Fill in the prompt template with the relevant documents and the question
    prompt = fact_checking_prompt.format(statement=statement, claim_reasoning_pair=reasoning)

    # claude output
    message = client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=1000,
        temperature=0,
        system=fact_checking_system,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    
    # Decode the generated text
    output_text = message.content[0].text

    try:
        verdict = output_text.split('Verdict:')[-1].split('Reasoning:')[0].strip()
        reasoning = output_text.split('Reasoning:')[-1].strip()
        return verdict, reasoning
    except:
        raise ValueError(f"Error parsing model output: {output_text}")

In [10]:
import os
path="/Users/sunnyfang/Library/CloudStorage/GoogleDrive-yf2610@barnard.edu/My Drive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/LLM-FactChecker"
os.chdir(path)
df = pd.read_csv('data/cleaned_pilot.csv')
claude_sample = pd.read_csv('eval_data/claude_complete.csv')

In [12]:
import pandas as pd
claude = pd.concat([df, claude_sample], axis = 1)

In [19]:
from tqdm.auto import tqdm

claim_only_results = []
for idx, row in tqdm(claude[:5].iterrows(), total = len(claude[:5])):
    statement = row['statement']
    reasoning = row['reasonings']
    claim_only_results.append(verify_statement_claude_claim_reasoning(statement, reasoning))

  0%|          | 0/5 [00:00<?, ?it/s]

In [22]:
for idx, row in tqdm(claude[5:10].iterrows(), total = len(claude[5:10])):
    statement = row['statement']
    reasoning = row['reasonings']
    claim_only_results.append(verify_statement_claude_claim_reasoning(statement, reasoning))

  0%|          | 0/5 [00:00<?, ?it/s]

In [26]:
for idx, row in tqdm(claude[10:15].iterrows(), total = len(claude[10:15])):
    statement = row['statement']
    reasoning = row['reasonings']
    claim_only_results.append(verify_statement_claude_claim_reasoning(statement, reasoning))

  0%|          | 0/5 [00:00<?, ?it/s]

In [29]:
for idx, row in tqdm(claude[15:20].iterrows(), total = len(claude[15:20])):
    statement = row['statement']
    reasoning = row['reasonings']
    claim_only_results.append(verify_statement_claude_claim_reasoning(statement, reasoning))

  0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
for idx, row in tqdm(claude[20:].iterrows(), total = len(claude[20:])):
    statement = row['statement']
    reasoning = row['reasonings']
    claim_only_results.append(verify_statement_claude_claim_reasoning(statement, reasoning))

  0%|          | 0/30 [00:00<?, ?it/s]

In [33]:
claude_claims_only = pd.DataFrame(data = {
    "verdicts": [x[0] for x in claim_only_results],
    "reasonings": [x[1] for x in claim_only_results]})

In [36]:
# modify base_dir
base_dir = '/Users/sunnyfang/Library/CloudStorage/GoogleDrive-yf2610@barnard.edu/My Drive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/'
filename = 'final-claude-3-sonnet-reasonings.xlsx'
full_path = os.path.join(base_dir, filename)
claude_claims_only.to_excel(full_path)

base_dir = '/Users/sunnyfang/Library/CloudStorage/GoogleDrive-yf2610@barnard.edu/My Drive/courses/AY 23-24/24 SP/COMS-W3997 LLM Foundations & Ethics/LLM Project/LLM-FactChecker/eval_data'
filename = 'claude_reasonings.csv'
full_path = os.path.join(base_dir, filename)
claude_claims_only.to_csv(full_path)